# case studies

## main objective
1. gather all low-performance cases for each group of approach
2. compare their results to each other for different answer types
    - improvements from no context -> RAG [how helpful is RAG?]
        - get QIDs with high answer-f1 from RAG results.
        - answer them with approach 1.
        - filter out the QIDs that has biggest difference in answer-f1 from both tables.
        - calculate the proportion, as improvement rate.
        - analyze: LLM parametric memory is not enough for domain-specific data. since context is required.
    - improvements from RAG -> gold [how important is the retrieval quality?]
        - get QIDs with low answer-f1 from RAG results.
        - locate the gold evidences based on the chosen "evidence_index".
        - prompt LLM to answer with those gold evidences as context.
        - calculate the proportion, as improvement rate.
        - analyze: LLM is misleaded by poo
    - degradation from RAG -> gold 
    - cases of low-recall but high-ansF1 in RAG 
        - fix the problem in the original evaluation script: calculate the percentage of annotator mismatch from evidence-F1 & answer-F1.
        - case study on the one only with aligned annotator.
        - if the mismatch percentage is high, discuss solutions with teacher.
3. research question (see if there is clue for improving the results):
    - significant improvement trend when switching from one approach to the other? 
    - common pattern on performance improvement/degradation among all approaches?

---
## group of approaches 
1. no-context: llama3-8b
2. RAG-context: SBERT+llama3-8b
3. gold-context: llama3-8b

---
## answer types
1. extractive
2. abstractive
3. boolean
4. unanswerable
5. overall
---
## TODOs
[ done ] prompt
- prepare for no-context approach
- revise for RAG & gold approach.

[ done ] evaluation results
- optimize evaluation script

In [ ]:
# system prompt (same for all approaches)
"You are an expert in NLP research field."

In [ ]:
# user prompts

# modified on HF playground
"""
Task: 
Given a question in NLP research field and some relevant snippets, provide an accurate and contextually relevant answer.
The answer can take one of the following forms:
- Boolean: 'Yes' or 'No.'
- Abstractive: A concise and synthesized answer after reasoning on both the question and the provided content.
- Extractive: A direct excerpt from the text.
- Unanswerable: If the question cannot be answered based on the provided content.

Input Details:
1. Paper Title: A Crowd-based Evaluation of Abuse Response Strategies in Conversational Agents
2. Relevant Snippets: 
We first gather abusive utterances from 600K conversations with US-based customers. We search for relevant utterances by simple keyword spotting and find that about 5% of the corpus includes abuse, with mostly sexually explicit utterances. Previous research reports even higher levels of abuse between 11% BIBREF2 and 30% BIBREF6. Since we are not allowed to directly quote from our corpus in order to protect customer rights, we summarise the data to a total of 109 “prototypical" utterances - substantially extending the previous dataset of 35 utterances from Amanda:EthicsNLP2018 - and categorise these utterances based on the Linguistic Society's definition of sexual harassment BIBREF7:
3. Question: How much data did they gather from crowdsourcing?

Constraints:
- Provide a direct and concise answer to the question based only on the relevant snippets.
- If the answer is unanswerable, state 'Unanswerable' explicitly. Otherwise, the answer to the given question must be enclosed with the special tokens '<answer>' and '</answer>'.
- Your answer should be as concise as possible.

Answer:
"""

## llama3.1-8b
(
    "Task: Given a research paper and a corresponding question derived from it, provide an accurate and contextually relevant answer. "
    "The answer can take one of the following forms:\n"
    "- Boolean: 'Yes' or 'No.'\n"
    "- Abstractive: A concise and synthesized answer after reasoning on both the question and the provided content.\n"
    "- Extractive: A direct excerpt from the text.\n"
    "- Unanswerable: If the question cannot be answered based on the provided content.\n\n"
    "Input Details:\n"
    f"1. Paper Title: {full_papers[paper_id]['title']}\n"
    f"2. Paper Contents:\n{full_papers[paper_id]['text']}\n"
    f"3. Question: {question_text}\n\n"
    "Provide a direct and concise answer to the question based only on the given paper's contents. "
    "If the answer is unanswerable, state 'Unanswerable' explicitly.\n\n"
    "Otherwise, the answer to the given question must be enclosed with '<<<' and '>>>'."
    "Your answer should be as concise as possible."
    "Answer:\n"
)


## llama3-8b, RAG/gold as context
concatenated_topk: str = "\n".join(all_topk_paras[question_id])
(
    "Task: Given a question in NLP research field and some relevant snippets, provide an accurate and contextually relevant answer. "
    "The answer can take one of the following forms:\n"
    "- Boolean: 'Yes' or 'No.'\n"
    "- Abstractive: A concise and synthesized answer after reasoning on both the question and the provided content.\n"
    "- Extractive: A direct excerpt from the text.\n"
    "- Unanswerable: If the question cannot be answered based on the provided content.\n\n"
    "Input Details:\n"
    f"1. Paper Title: {paper_title}\n"
    f"2. Relevant Snippets:\n{concatenated_topk}\n"
    f"3. Question: {question_text}\n\n"
    "Provide a direct and concise answer to the question based only on the relevant snippets. "
    "If the answer is unanswerable, state 'Unanswerable' explicitly.\n\n"
    "Otherwise, the answer to the given question must be enclosed with the special tokens '<answer>' and '</answer>'."
    "Your answer should be as concise as possible."
    "Answer:\n"
)


# llama3-8b, no context
(
    "Task: Given a question from NLP research field and the title of the relevant research paper, provide an accurate and contextually relevant answer. "
    "The answer can take one of the following forms:\n"
    "- Boolean: 'Yes' or 'No.'\n"
    "- Abstractive: A concise and synthesized answer after reasoning on both the question and the provided content.\n"
    "- Extractive: A direct excerpt from the text.\n"
    "- Unanswerable: If the question cannot be answered based on the provided content.\n\n"
    "Input Details:\n"
    f"1. Paper Title: {paper_title}\n"
    f"2. Question: {question_text}\n\n"
    "Provide a direct and concise answer to the question using the best of your knowledge in the field."
    "If the answer is unanswerable, state 'Unanswerable' explicitly.\n\n"
    "Otherwise, the answer to the given question must be enclosed with the special tokens '<answer>' and '</answer>'."
    "Your answer should be as concise as possible."
    "Answer:\n"
)

NameError: name 'full_papers' is not defined

### extractive cases

1. no annotated data
    - compute total percentage
    - analyze:
        - should it be considered exception?
        - if not, how did the LLM perform?

2. single evidence, low ans-f1, low ev-f1
    - compute total percentage
    - filter out QIDs where chosen annotator ids match
    - replace context with gold evidences provided by the same annotator
    - analyze:
        - [2a] better answer-f1: how helpful is the gold evidence?
        - [2b] worse or same answer-f1: why can't LLM understand the gold evidence?

3. single evidence, low ans-f1, high ev-f1
    - compute total percentage = 0%
    - filter out QIDs where chosen annotator ids match
    - analyze:
        - is it hard for LLM to understand the given context? (link the conclusion with [2b])

4. multi evidence (n>10, outliers), low ans-f1, low ev-f1
    - compute percentage
    - analyze:
        - why are there so many gold paras?

5. multi evidence (3 < n < 10, truncated), low ans-f1, low ev-f1
    - compute percentage
    - analyze:
        - is the low ans-f1 due to incomplete context? (only top 3 is selected when it needs more than 3 paras to deduce the answer)

6. multi evidence (n=2), low ans-f1, low ev-f1
    - compute total percentage
    - filter out QIDs where chosen annotator ids match
    - replace context with gold evidences provided by the same annotator
    - analyze:
        - [6a] better answer-f1: how helpful is the gold evidence?
        - [6b] worse or same answer-f1: why can't LLM understand the gold evidence? is the relationship between evidences too complicated?

7. multi evidence (n=2), low ans-f1, high ev-f1
    - compute total percentage
    - filter out QIDs where chosen annotator ids match
    - analyze:
        - [7a]
        - [7b] is it hard for LLM to understand the given context? (link the conclusion with [6b])

In [ ]:
# criterias to determine a good and bad answer
## baseline: llama3.1-8b (zero-shot)
min_ans_f1 = 0.46

## heuristic settings
min_ev_f1 = 0.4
max_ev_f1 = 0.7

In [1]:
i = [1, 2]
j = [3, 4]
print(i+j)

[1, 2, 3, 4]


In [ ]:
# case 1
# only 1 entry
qid = "788f70a39c87abf534f4a9ee519f6e5dbf2543c2"

In [9]:
# rerun with llama3-8b
import my_evaluator

# case 2b
qid = "e42916924b69cab1df25d3b4e6072feaa0ba8084"

# case 3b
qid = "f64449a21c452bc5395a0f0a49fb49825e6385f4"
print(my_evaluator.token_f1_score("600K", "600K conversations"))

# case 5b
qid = "bc8526d4805e2554adb2e9c01736d3f3a3b19895"
print(my_evaluator.token_f1_score("LDA, Doc-NADE, HTMM, GMNTM", "The baselines used in their experiments include LDA, Doc-NADE, HTMM, and GMNTM."))

# case 6b
qid = "67cb001f8ca122ea859724804b41529fea5faeef"
print(my_evaluator.token_f1_score("ECNU, HCTI", "The state-of-the-art methods they compare with are Glove, SL999, and HCTI BIBREF5 for sentence modeling."))

# case 6d
qid = "516b691ef192f136bb037c12c3c9365ef5a6604c"
print(my_evaluator.token_f1_score("in this paper we proposed a Gated Recurrent Neural Tensor Network (GRURNTN) to combine these two advantages into an RNN architecture. In this architecture, the tensor product operation is applied between the current input and previous hidden layer multiplied by the reset gates for calculating the current candidate hidden layer values., As with GRURNTN, we also applied the tensor product operation for the LSTM unit to improve its performance. In this architecture, the tensor product operation is applied between the current input and the previous hidden layers to calculate the current memory cell. The calculation is parameterized by the tensor weight. We call this architecture a Long Short Term Memory Recurrent Neural Tensor Network (LSTMRNTN).", "The introduced model combines the two factors by applying the tensor product operation between the current input and previous hidden layers or the reset gates, which is parameterized by tensor weights."))

0.6666666666666666
0.5333333333333333
0.125
0.28571428571428575


In [ ]:
case_study = ["e42916924b69cab1df25d3b4e6072feaa0ba8084", "f64449a21c452bc5395a0f0a49fb49825e6385f4", "bc8526d4805e2554adb2e9c01736d3f3a3b19895", "67cb001f8ca122ea859724804b41529fea5faeef", "516b691ef192f136bb037c12c3c9365ef5a6604c"]

In [4]:
import my_evaluator
# case 4
qid = "87c00edc497274ae6a972c3097818de85b1b384f"
predicted_evidences = ["The result of program $\\Pi _2$ is an one-to-one mapping of some of the words in the sentence into the importaint components of a sentence, called main components, i.e. subject, object and verb. The mapping is constructed by using the core arguments in Universal Dependency Relations . Since not every word in the sentence is in a core argument relation, there are some words in the sentence that are not in the domain of the mapping that $\\Pi _2$ produces. We denote these words are complement components. To identify these words, we encode the Non-core dependents and Nominal dependents from Universal Dependency Relations into the set of rules in program $\\Pi _3$.", "The input of Program $\\Pi _3$ is the position ($pos$) of the word in the sentence. Program $\\Pi _3$ is called whenever there is a new complement component discovered. That way of recursive calls is to identify the maximal chunk of the words that support the main components of the sentence. The result of this module is a list of vocabularies for the next steps.", "We begin with recognizing the main words (components) that play the most important roles in the sentence based on a given sentence structure. This is achieved by program $\\Pi _2$ (Listing ). The first four rules of $\\Pi _2$ determine the main subject and verb of the sentence whose structure is #1, #2, #3, or #5. Structure #4 requires a special treatment since the components following tobe can be of different forms. For instance, in “Cathy is gorgeous,” the part after tobe is an adjective, but in “Cathy is a beautiful girl,” the part after tobe is a noun, though, with adjective beautiful. This is done using the four last rules of $\\Pi _2$."]
gold_evidences = [
  "To generate a sentence, we need a sentence structure and vocabularies. Our system is developed to emulate the process of a person learning a new language and has to make guesses to understand new sentences from time to time. For example, someone, who understands the sentence \u201cBill plays a game\u201d would not fully understand the sentence \u201cBill plays a popular board game\u201d without knowing the meaning of \u201cpopular\u201d and \u201cboard game\u201d but could infer that the latter sentence indicates that its subject plays a type of game.",
  "The overall design of our system is given in Figure FIGREF7. Given a paragraph, our system produces a GF program (a pair of an abstract and a concrete syntax), which can be used for sentence generation. The system consists of two components, understanding sentences and generating GF grammar. The first component is divided into two sub-components, one for recognizing the sentence structure and one for recognizing the sentence components. The second component consists of a GF grammar encoder and a GF grammar exporter. The encoder is responsible for generating a GF grammar for each sentence, while the exporter aggregates the grammars generated from the encoder, and produces a comprehensive grammar for the whole paragraph.",
  "Method ::: Sentence Structure Recognition",
  "The sentence structure recognition process involves 2 modules: natural language processing (NLP) module and logical reasoning on result from NLP module. In this paper, we make use of the Stanford Parser tools described in BIBREF10, BIBREF11, BIBREF12, BIBREF13, BIBREF14",
  "The NLP module tokenizes the input free text to produce a dependency-based parse tree and part-of-speech tag (POS tag). The dependency-based parse tree and the POS tag are then transform into an answer set program (ASP) BIBREF15 which contains only facts. Table TABREF13 shows the transformation of the result of NLP module into an ASP program for the sentence \u201cBill plays a game\u201d. In this table, nsubj, det, dobj and punct denote relations in the dependency-based parse tree, and mean nominal subject, determiner, direct object and punctuation respectively. Full description of all relations in a dependency-based parse tree can be found in the Universal Dependency website. The second set of notations are the POS tag PRP, VBP, DT and NN corresponding to pronoun, verb, determiner and noun. Readers can find the full list of POS tag in Penn Treebank Project.",
  "From the collection of the dependency atoms from the dependency-based parse tree, we determine the structure of a sentence using an ASP program, called $\\Pi _1$ (Listing ).",
  "Each of the rule above can be read as if the right-hand side is true then the left-hand side must be true. These rules define five possible structures of a sentence represented by the atom structure(x,y). $x$ and $y$ in the atom structure(x,y) denote the type of the structure and the number of dependency relations applied to activate the rule generating this atom, respectively. We refer to $y$ as the $i$-value of the structure. For example, $structure(1,1)$ will be recognized if the nsubj relation is in the dependency-based parse tree; $structure(3,3)$ needs 3 dependency relations to be actived: nsubj, xcomp and dobj. We often use structure #$x$ to indicate a structure of type $x$.",
  "Together with the collection of the atoms encoding the relations in the dependency-based parse tree, $\\Pi _1$ generates several atoms of the form $structure(x,y)$ for a sentence. Among all these atoms, an atom with the highest $i$-value represents the structure constructed using the highest number of dependency relations. And hence, that structure is the most informative structure that is recoginized for the sentence. Observe that $structure(1,1)$ is the most simplified structure of any sentence.",
  "Method ::: Sentence Components Recognition",
  "The goal of this step is to identify the relationship between elements of a sentence structure and chunks of words in a sentence from the POS tags and the dependency-based parse tree. For example, the sentence \u201cBill plays a game\u201d is encoded by a structure #2 and we expect that Bill, plays, and game correspond to the subject, verb, and object, respectively.",
  "We begin with recognizing the main words (components) that play the most important roles in the sentence based on a given sentence structure. This is achieved by program $\\Pi _2$ (Listing ). The first four rules of $\\Pi _2$ determine the main subject and verb of the sentence whose structure is #1, #2, #3, or #5. Structure #4 requires a special treatment since the components following tobe can be of different forms. For instance, in \u201cCathy is gorgeous,\u201d the part after tobe is an adjective, but in \u201cCathy is a beautiful girl,\u201d the part after tobe is a noun, though, with adjective beautiful. This is done using the four last rules of $\\Pi _2$.",
  "The result of program $\\Pi _2$ is an one-to-one mapping of some of the words in the sentence into the importaint components of a sentence, called main components, i.e. subject, object and verb. The mapping is constructed by using the core arguments in Universal Dependency Relations . Since not every word in the sentence is in a core argument relation, there are some words in the sentence that are not in the domain of the mapping that $\\Pi _2$ produces. We denote these words are complement components. To identify these words, we encode the Non-core dependents and Nominal dependents from Universal Dependency Relations into the set of rules in program $\\Pi _3$.",
  "Program $\\Pi _3$ (Listing ), together with the atoms extracted from the dependency-based parse tree such as $compound(P,N)$ ($N$ is compound noun at the position $P$ in the sentence), $amod(P,J)$ ($J$ is an adjective modifier), etc., is used to identify the complement components of the main components computed by $\\Pi _2$ while maintaining the structure of the sentence created by $\\Pi _1$. For example, a complement of a noun could be another noun (as \u201cboard\u201d in \u201cboard game\u201d), or an adjective (as \u201cpopular\u201d in \u201cpopular board game\u201d), or a preposition (as \u201cfor adults\u201d in \u201cboard game for adults\u201d).",
  "The input of Program $\\Pi _3$ is the position ($pos$) of the word in the sentence. Program $\\Pi _3$ is called whenever there is a new complement component discovered. That way of recursive calls is to identify the maximal chunk of the words that support the main components of the sentence. The result of this module is a list of vocabularies for the next steps.",
  "Method ::: GF Grammar Encoder",
  "The goal of the encoder is to identify appropriate GF rules for the construction of a GF grammar of a sentence given its structure and its components identified in the previous two modules. This is necessary since a sentence can be encoded in GF by more than one set of rules; for example, the sentence \u201cBill wants to play a game\u201d can be encoded by the rules",
  "In GF, NP, VV, V2, VP, and Cl stand for noun phrase, verb-phrase-complement verb, two-place verb, verb phrase and clause, respectively. Note that although the set of GF grammatical rules can be used to construct a constituency-based parse tree , the reverse direction is not always true. To the best of our knowledge, there exists no algorithm for converting a constituency-based parse tree to a set GF grammar rules. We therefore need to identify the GF rules for each sentence structure.",
  "In our system, a GF rule is assigned to a structure initially (Table TABREF19). Each rule in Table TABREF19 represents the first level of the constituency-based parse tree. It acts as the coordinator for all other succeeding rules.",
  "Given the seed components identified in Section SECREF15 and the above GF rules, a GF grammar for each sentence can be constructed. However, this grammar can only be used to generate fairly simple sentences. For example, for the sentence \u201cBill plays a popular board game with his close friends.\u201d, a GF grammar for structure #2 can be constructed, which can only generate the sentence \u201cBill plays game.\u201d because it does not contain any complement components identified in Section SECREF15. Therefore, we assgin a set of GF rules for the construction of each parameter in the GF rules in Table TABREF19. The set of GF rules has to follow two conventions. The first one is after applying the set of rules to some components of the sentence, the type of the production is one of the type in Table TABREF19, e.g. $NP$, $VP$, $Cl$, $V2$, .... The second convention is that the GF encoder will select the rules as the order from top to bottom in Table TABREF20. Note that the encoder always has information of what type of input and output for the rule it is looking for.",
  "For instance, we have \u201cgame\u201d is the object (main components), and we know that we have to construct \u201cgame\u201d in the result GF grammar to be a NP (noun phrase). Program $\\Pi _2$ identifies that there are two complement components for the word \u201cgame\u201d, which are \u201cboard\u201d and \u201cpopular\u201d, a noun and an adjective respectively. The GF encoder then select the set of rules: N $\\rightarrow $ N $\\rightarrow $ CN and A $\\rightarrow $ AP to create the common noun \u201cboard game\u201d and the adjective phrase first. The next rule is AP $\\rightarrow $ CN $\\rightarrow $ CN. The last rule to be applied is CN $\\rightarrow $ NP. The selection is easily decided since the input and the output of the rules are pre-determined, and there is no ambiguity in the selection process.",
  "The encoder uses the GF rules and the components identified by the previous subsections to produce different constructors for different components of a sentence. A part of the output of the GF encoder for the object \u201cgame\u201d is",
  "The encoder will also create the operators that will be included in the oper section of the GF grammar for supporting the new constructor. For example, the following operators will be generated for serving the Game constructor above:",
  "Method ::: GF Grammar Exporter",
  "The GF Grammar Exporter has the simplest job among all modules in the system. It creates a GF program for a paragraph using the GF grammars created for the sentences of the paragraph. By taking the union of all respective elements of each grammar for each sentence, i.e., categories, functions, linearizations and operators, the Grammar Exporter will group them into the set of categories (respectively, categories, functions, linearizations, operators) of the final grammar."
  ]
print(len(gold_evidences))
print(len(set(gold_evidences)))
print(my_evaluator.paragraph_f1_score(predicted_evidences, gold_evidences))

24
24
0.2222222222222222


In [ ]:
# case 5b
qid = "bc8526d4805e2554adb2e9c01736d3f3a3b19895"
gold = [
    "The following baselines were used in our experiments:",
    "LDA BIBREF2 . LDA is the classic topic model, and we used GibbsLDA++ for its implementation.",
    "Doc-NADE BIBREF24 . Doc-NADE is a representative neural network based topic model. We used the open-source code provided by the authors.",
    "HTMM BIBREF9 . HTMM models consider the sentence level Markov transitions. Similar to Doc-NADE, the implementation was provided by the authors.",
    "GMNTM BIBREF12 . GMNTM considers models the order of words within a sentence by a feedforward neural network. We implemented GMNTM according the descriptions in their papers by our own."
    ]
for p in gold:
    print(p)

The following baselines were used in our experiments:
LDA BIBREF2 . LDA is the classic topic model, and we used GibbsLDA++ for its implementation.
Doc-NADE BIBREF24 . Doc-NADE is a representative neural network based topic model. We used the open-source code provided by the authors.
HTMM BIBREF9 . HTMM models consider the sentence level Markov transitions. Similar to Doc-NADE, the implementation was provided by the authors.
GMNTM BIBREF12 . GMNTM considers models the order of words within a sentence by a feedforward neural network. We implemented GMNTM according the descriptions in their papers by our own.


# case 6c

### abstractive
#### criteria
- worst case [comparing with QASPER LED baseline] - answer-F1 < 7.38 (test split)
- reasonable case [comparing with llama3.1(8B)] - 7.38 < answer-F1 < 27.82

In [ ]:
# case 3
non